In [1]:
# import necessary libraries
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
conda install -c conda-forge bs4 --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - bs4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.1       |   py36h9f0ad1d_0         163 KB  conda-forge
    bs4-4.9.1                  |                0           4 KB  conda-forge
    soupsieve-2.0.1            |   py36h9f0ad1d_0          56 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         223 KB

The following NEW packages will be INSTALLED:

  beautifulsoup4     conda-forge/linux-64::beautifulsoup4-4.9.1-py36h9f0ad1d_0
  bs4                conda-forge/noarch::bs4-4.9.1-0
  soupsieve          conda-forge/linux-64::soupsieve-2.0.1-py36h9f0ad1d_0



beautifulsoup4-4.9.1 | 163 KB    | ##################################### | 100% 
bs4-4.9.1 

In [3]:
import urllib.request         # import the library we use to open URLs
from bs4 import BeautifulSoup # import the BeautifulSoup library so we can parse HTML and XML documents

# specify which URL/web page we are going to be scraping
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)

# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, 'html5lib')

#BeautifulSoup function 'prettify' to look at the HTML our chosen web page
#print(soup.prettify())

In [4]:
# use the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable
all_tables=soup.find_all("table")
#all_tables

# class id is ”wikitable sortable”, use this to get Beautifulsoup to only bring back the table data and keep in a variable ”right_table“
right_table=soup.find('table', class_='wikitable sortable')
#right_table

In [5]:
# set up 3 empty lists to store data from 3 columns in Wikipedia
A=[]
B=[]
C=[]

for row in right_table.find_all('tr'):     # BeautifulSoup ‘find_all’ function to look for the string ‘tr’ for each row 
    cells = row.find_all('td')             # find_all again to search each row for <td> tags with the ‘td’ string add all of these to a variable called ‘cells’ 
    if len(cells)==3:
        A.append(cells[0].find(text=True)) # If there are then use find(text=True)) option to extract the content string from within each <td>
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [6]:
# create dataframe and assigning each list A-C into column with the name of table columns
df = pd.DataFrame(A, columns=['Postal Code'])   
df['Borough']=B
df['Neighborhood']=C
df.head()

,Postal Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [7]:
# remove '\n' character in each cells
df = df.replace('\n','',regex=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [8]:
# replace "Not assigned" with "NaN" in Borough and Neighborhood column
df.replace('Not assigned', np.nan, inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [9]:
# drop whole rows with Borough that NaN in cells
df.dropna(subset=["Borough"], axis=0, inplace=True)

# reset index because several drop rows
df.reset_index(drop=True, inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
# detecting missing data
missing_data = df.isnull()
missing_data.head()

,Postal Code,Borough,Neighborhood
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False


In [11]:
# count missing values in each column
for column in missing_data.columns.values.tolist():
    print (column)
    print (missing_data[column].value_counts())
    print ("")

Postal Code
False    103
Name: Postal Code, dtype: int64

Borough
False    103
Name: Borough, dtype: int64

Neighborhood
False    103
Name: Neighborhood, dtype: int64



In [12]:
# check number of rows of dataframe
df.shape[0]

103

In [13]:
# read file csv that contains latitude & longitude based on Postal Code
df1 = pd.read_csv('http://cocl.us/Geospatial_data')
df1.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
# with merge function, df and df1 in this case `Postal Code` is the only column name in  both dataframes
merged_inner = pd.merge(left=df, right=df1, left_on='Postal Code', right_on='Postal Code')
merged_inner.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
